41. 係り受け解析結果の読み込み（文節・係り受け）

40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

    def __str__(self):
        return f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}"


In [2]:
class Chunk:
    def __init__(self, morphs=None, dst=-1):
        self.morphs = morphs if morphs else []  # Morphオブジェクトのリスト
        self.dst = dst  # 係り先文節インデックス番号
        self.srcs = []  # 係り元文節インデックス番号のリスト

    def __str__(self):
        morphs_surface = ''.join([morph.surface for morph in self.morphs])
        return f"Chunk: {morphs_surface}, dst: {self.dst}, srcs: {self.srcs}"


In [4]:
def parse_cabocha_output(parsed_file):
    sentences = []
    with open(parsed_file, 'r', encoding='utf-8') as f:
        chunks = {}
        chunk = None
        for line in f:
            if line == 'EOS\n':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                if chunks:
                    sorted_chunks = sorted(chunks.items())
                    sentence = [chunk for idx, chunk in sorted_chunks]
                    for idx, chunk in sorted_chunks:
                        if chunk.dst != -1:
                            sentence[chunk.dst].srcs.append(idx)
                    sentences.append(sentence)
                chunks = {}
                chunk = None
            elif line[0] == '*':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                cols = line.split(' ')
                chunk_idx = int(cols[1])
                dst = int(cols[2].rstrip('D'))
                chunk = Chunk(dst=dst)
            else:
                surface, feature = line.split('\t')
                features = feature.split(',')
                morph = Morph(surface, features[6], features[0], features[1])
                chunk.morphs.append(morph)
    return sentences


In [5]:
def display_first_sentence(parsed_file):
    sentences = parse_cabocha_output(parsed_file)
    if sentences:
        first_sentence = sentences[0]
        for chunk in first_sentence:
            chunk_text = ''.join([morph.surface for morph in chunk.morphs])
            print(f"Chunk: {chunk_text}, Dst: {chunk.dst}")

parsed_file = 'ai.ja.txt.parsed'
display_first_sentence(parsed_file)


Chunk: 人工知能, Dst: 17
Chunk: （じんこうちのう、、, Dst: 17
Chunk: AI, Dst: 3
Chunk: 〈エーアイ〉）とは、, Dst: 17
Chunk: 「『計算, Dst: 5
Chunk: （）』という, Dst: 9
Chunk: 概念と, Dst: 9
Chunk: 『コンピュータ, Dst: 8
Chunk: （）』という, Dst: 9
Chunk: 道具を, Dst: 10
Chunk: 用いて, Dst: 12
Chunk: 『知能』を, Dst: 12
Chunk: 研究する, Dst: 13
Chunk: 計算機科学, Dst: 14
Chunk: （）の, Dst: 15
Chunk: 一分野」を, Dst: 16
Chunk: 指す, Dst: 17
Chunk: 語。, Dst: -1
